<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Unit%208%20Aquifer%20Influx/notebook/8_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 8 Aquifer Influx**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [1]:
!git clone https://www.github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 686 (delta 68), reused 0 (delta 0), pack-reused 535
Receiving objects: 100% (686/686), 3.08 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (291/291), done.
